# Trade accounting

In [7]:
import numpy as np
import pandas as pd
import os
import re
import duckdb

In [8]:
inputfolder = 'ADB-MRIO'
outputfileroot = 'ta'
version = None

filelist = [file for file in os.listdir(f'../data/interim/{inputfolder}') if not file.startswith('.')]
filelist.sort()

## Setup

### Parameters

In [9]:
sectors = pd.read_excel('../data/interim/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)

G = 73      # Number of countries + ROW
N = 35      # Number of sectors
f = 5       # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

### Helper functions

In [10]:
def ind(s, i=None, G=G, N=N):
    '''
    Selects the indices of country s in any GxN array. If inverse=True, selects all indices but country s.
    Optional argument i selects a specific element in the array.
    '''
    ind_s = list(range((abs(s) - 1) * N, abs(s) * N))
    if s<0:
        ind_s = np.setdiff1d(np.arange(G*N), ind_s)

    if i is not None:
        return ind_s[i]
    else:
        return ind_s

In [11]:
def asvector(matrix):
    ''' 
    Flattens a matrix into a column vector by taking the first column, then appending the second column,
    and so on.
    '''
    vector = np.reshape(matrix, (-1, ), order='F')
    return vector

In [12]:
def zeroout(matrix, row=None, col=None, inverse=False, G=G):
    '''
    Zeroes out the NxN block diagonals of a GNxGN matrix. If inverse=True, all elements but the NxN block
    diagonals are zeroed out. An arbitrary NxN block can be zeroed out by passing ind() in the row and col 
    arguments. 
    '''
    Nrow = matrix.shape[0] // G
    Ncol = matrix.shape[1] // G
    zeroed_matrix = np.copy(matrix)
    
    if row is not None and col is not None:
        zeroed_matrix[np.ix_(row, col)] = 0
    else:
        for k in range(G):
            zeroed_matrix[(k*Nrow):(k*Nrow)+Nrow, (k*Ncol):(k*Ncol)+Ncol] = 0
    
    if inverse:
        return matrix - zeroed_matrix
    else:
        return zeroed_matrix

In [13]:
def diagvec(vector, N=N):
    '''
    Splits a vector into N-sized vectors and arranges them in a block diagonal matrix. 
    '''
    length = vector.shape[0]
    G = length // N
    matrix = np.zeros((length, G))
    for k in range(G):
        matrix[k*N:(k+1)*N, k] = vector[k*N:(k+1)*N]
    return matrix

In [14]:
def diagmat(matrix, offd=False, N=N):
    '''
    Reshapes a matrix into an appropriate block diagonal matrix. 
    '''
    nrow = matrix.shape[0]
    G = nrow // N
    
    if offd:
        for k in range(G):
            matrix[k*N:(k+1)*N, k] = 0
        vector = np.sum(matrix, axis=1)
    else:
        vector = []
        for k in range(G):
            vector.extend(matrix[k*N:(k+1)*N, k]) 
        vector = np.array(vector)
    
    return diagvec(vector)

In [15]:
def diagrow(vector, N=N):
    G = vector.shape[0] // N
    matrix = np.empty((N,0))
    for k in range(G):
        matrix = np.hstack((matrix, np.diag(vector[k*N:(k+1)*N])))
    return matrix

## Decompositions

Note that the country index `s` used henceforth corresponds to the MRIO country indices, which start at 1 and not 0.

### Country level

In [457]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/final/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    zuse = np.sum(Z, axis=0)
    zsales = np.sum(Z, axis=1)
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd = zeroout(Y, inverse=True)
    Yf = zeroout(Y)
    v = np.where(x != 0, va / x, 0)
    Dx = np.diag(np.where(x != 0, 1 / x, 0))
    A = Z @ Dx
    Ad = zeroout(A, inverse=True)
    Af = zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        Exports = np.sum(E[ind(s)][:, np.arange(G) != s-1], axis=0)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, row=ind(s), col=ind(-s)))
        VB_DC = v[ind(s)] @ Bnots[np.ix_(ind(s), ind(s))]
        VB_FC = v[ind(-s)] @ Bnots[np.ix_(ind(-s), ind(s))]
        DAVAX1 = VB_DC @ Y[ind(s)][:, np.arange(G) != s-1]
        DAVAX2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ Yd[ind(-s)][:, np.arange(G) != s-1]
        REX1 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(np.sum(Yf[ind(-s)][:, np.arange(G) != s-1], axis=1))
        REX2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagmat(Af[ind(-s), :] @ B @ Y[:, np.arange(G) != s-1], offd=True)
        REX3 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagmat(Af[ind(-s), :] @ B @ Y[:, np.arange(G) != s-1])
        REF1 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(Yf[ind(-s)][:, s-1])
        REF2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(Af[ind(-s), :] @ B @ Y[:, s-1])
        FVA = VB_FC @ E[ind(s)][:, np.arange(G) != s-1]
        PDC1 = VB_DC @ Af[ind(s), :] @ B[:, ind(s)] @ E[ind(s)][:, np.arange(G) != s-1]
        PDC2 = VB_FC @ Af[ind(s), :] @ B[:, ind(s)] @ E[ind(s)][:, np.arange(G) != s-1]

        DFs = pd.DataFrame({
            't': year,
            's': s,
            'r': np.setdiff1d(np.arange(1, G+1), s),
            'Exports': Exports,
            'DAVAX1': DAVAX1,
            'DAVAX2': DAVAX2,
            'REX1': REX1,
            'REX2': REX2,
            'REX3': REX3,
            'REF1': REF1,
            'REF2': REF2,
            'FVA': FVA,
            'PDC1': PDC1,
            'PDC2': PDC2
        })
        DF = pd.concat([DF, DFs], ignore_index=True)

    print(f'{year} done')

if version is None:
    outputfilename = f'{outputfileroot}.parquet'
else:
    outputfilename = f'{outputfileroot}_{version}.parquet'

DF.to_parquet(f'../data/interim/trade-accounting/{outputfilename}', index=False)

2017 done
2018 done
2019 done
2020 done
2021 done


### By export sectors

In [456]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/final/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    zuse = np.sum(Z, axis=0)
    zsales = np.sum(Z, axis=1)
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd = zeroout(Y, inverse=True)
    Yf = zeroout(Y)
    v = np.where(x != 0, va / x, 0)
    Dx = np.diag(np.where(x != 0, 1 / x, 0))
    A = Z @ Dx
    Ad = zeroout(A, inverse=True)
    Af = zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        Exports = E[ind(s)][:, np.arange(G) != s-1]
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, row=ind(s), col=ind(-s)))
        VB_DC = np.diag(v[ind(s)] @ Bnots[np.ix_(ind(s), ind(s))])
        VB_FC = np.diag(v[ind(-s)] @ Bnots[np.ix_(ind(-s), ind(s))])
        DAVAX1 = VB_DC @ Y[ind(s)][:, np.arange(G) != s-1]
        DAVAX2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ Yd[ind(-s)][:, np.arange(G) != s-1]
        REX1 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(np.sum(Yf[ind(-s)][:, np.arange(G) != s-1], axis=1))
        REX2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagmat(Af[ind(-s), :] @ B @ Y[:, np.arange(G) != s-1], offd=True)
        REX3 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagmat(Af[ind(-s), :] @ B @ Y[:, np.arange(G) != s-1])
        REF1 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(Yf[ind(-s)][:, s-1])
        REF2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(Af[ind(-s), :] @ B @ Y[:, s-1])
        FVA = VB_FC @ E[ind(s)][:, np.arange(G) != s-1]
        PDC1 = np.diag(v[ind(s)] @ Bnots[np.ix_(ind(s), ind(s))] @ Af[ind(s), :] @ B[:, ind(s)]) @ E[ind(s)][:, np.arange(G) != s-1]
        PDC2 = np.diag(v[ind(-s)] @ Bnots[np.ix_(ind(-s), ind(s))] @ Af[ind(s), :] @ B[:, ind(s)]) @ E[ind(s)][:, np.arange(G) != s-1]

        DFs = pd.DataFrame({
            't': year,
            's': s,
            'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1),
            'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1),
            'REX2': asvector(REX2),
            'REX3': asvector(REX3),
            'REF1': asvector(REF1),
            'REF2': asvector(REF2),
            'FVA': asvector(FVA),
            'PDC1': asvector(PDC1),
            'PDC2': asvector(PDC2),
        })
        DF = pd.concat([DF, DFs], ignore_index=True)

    print(f'{year} done')

if version is None:
    outputfilename = f'{outputfileroot}-es.parquet'
else:
    outputfilename = f'{outputfileroot}-es_{version}.parquet'

DF.to_parquet(f'../data/interim/trade-accounting/{outputfilename}', index=False)

2017 done
2018 done
2019 done
2020 done
2021 done


### By origin sectors

In [16]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/interim/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    zuse = np.sum(Z, axis=0)
    zsales = np.sum(Z, axis=1)
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd = zeroout(Y, inverse=True)
    Yf = zeroout(Y)
    v = np.where(x != 0, va / x, 0)
    Dx = np.diag(np.where(x != 0, 1 / x, 0))
    A = Z @ Dx
    Ad = zeroout(A, inverse=True)
    Af = zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        Exports = E[ind(s)][:, np.arange(G) != s-1]
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, row=ind(s), col=ind(-s)))
        VB_DC = np.diag(v[ind(s)]) @ Bnots[np.ix_(ind(s), ind(s))]
        VB_FC = diagrow(v[ind(-s)]) @ Bnots[np.ix_(ind(-s), ind(s))]
        DAVAX1 = VB_DC @ Y[ind(s)][:, np.arange(G) != s-1]
        DAVAX2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ Yd[ind(-s)][:, np.arange(G) != s-1]
        REX1 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(np.sum(Yf[ind(-s)][:, np.arange(G) != s-1], axis=1))
        REX2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagmat(Af[ind(-s), :] @ B @ Y[:, np.arange(G) != s-1], offd=True)
        REX3 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagmat(Af[ind(-s), :] @ B @ Y[:, np.arange(G) != s-1])
        REF1 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(Yf[ind(-s)][:, s-1])
        REF2 = VB_DC @ A[np.ix_(ind(s), ind(-s))] @ Bd[np.ix_(ind(-s), ind(-s))] @ diagvec(Af[ind(-s), :] @ B @ Y[:, s-1])
        FVA = VB_FC @ E[ind(s)][:, np.arange(G) != s-1]
        PDC1 = VB_DC @ Af[ind(s), :] @ B[:, ind(s)] @ E[ind(s)][:, np.arange(G) != s-1]
        PDC2 = VB_FC @ Af[ind(s), :] @ B[:, ind(s)] @ E[ind(s)][:, np.arange(G) != s-1]

        DFs = pd.DataFrame({
            't': year,
            's': s,
            'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1),
            'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1),
            'REX2': asvector(REX2),
            'REX3': asvector(REX3),
            'REF1': asvector(REF1),
            'REF2': asvector(REF2),
            'FVA': asvector(FVA),
            'PDC1': asvector(PDC1),
            'PDC2': asvector(PDC2),
        })
        DF = pd.concat([DF, DFs], ignore_index=True)

    print(f'{year} done')

if version is None:
    outputfilename = f'{outputfileroot}-os.parquet'
else:
    outputfilename = f'{outputfileroot}-os_{version}.parquet'

DF.to_parquet(f'../data/interim/trade-accounting/{outputfilename}', index=False)

2017 done
2018 done
2019 done
2020 done
2021 done
